# F# Whistle API (version 2)

This is v2 of [WhistleAPI-DOTNET](https://github.com/aolney/WhistleAPI-DOTNET).
The original version was in C# and used WebClient.
For some reason, it just stopped working.

The same calls work in F# using [Fsharp.Data](https://fsharp.github.io/FSharp.Data/), so I decided to scrap the C# version and create this notebook with the minimal F# code needed to access the API.

The original code was informed by [an old description of the unofficial API](http://jared.wuntu.org/whistle-dog-activity-monitor-undocumented-api/).
In this new version, I loaded up [Charles Proxy](https://www.charlesproxy.com/) to [intercept encrypted traffic](https://medium.com/@hackupstate/using-charles-proxy-to-debug-android-ssl-traffic-e61fc38760f7) between the [Whistle Legacy app](https://play.google.com/store/apps/details?id=com.whistle.WhistleApp&hl=en_US) and the API endpoints.
I didn't notice any differences that were breaking, though it seems the app is sending additional information in the headers that's not needed to make the API work.

The entire API has not been implemented; that would be extremely tedious.
However, unlike v1, I'm providing a semi-complete domain model, using [FSharp.Json](https://vsapronov.github.io/FSharp.Json/) for serialization.
The API below represents what I consider to be the core interesting API.
If you want more, then see the comment in `AuthenticatedGet` to return raw JSON.

## API

In [71]:
#r "/z/aolney/repos/FSharp.Data.3.1.1/lib/net45/FSharp.Data.dll"
#r "/z/aolney/repos/FSharp.Json.0.4.0/lib/netstandard2.0/FSharp.Json.dll"

open FSharp.Data
open FSharp.Json

// ---- Domain model -----------------------------------------

type Authentication =
    {
        email : string
        password : string
    }
    
type Token = { token : string }
    
type Dog =
    {
        id: string
        city_id: string
        date_of_birth: string
        gender: string
        name: string
        weight: float
        weight_type: string
        age_in_months: int
        age_in_years: int
        breed_id: int
        breed_name: string
        average_minutes_active: float
        average_minutes_rest: float
        average_calories: float
        average_distance: float
        distance_units: string
        location : string
        device_id : string
        device_serial : string
    }
    
type User =
    {
        id : string
        first_name: string
        last_name :string
        email : string
        created_at : string
    }

type Device =
    {
        battery_days_left:  int
        battery_level:  float
        charging:  bool
        device_on:  bool
        firmware_version: string
        last_check_in:  string
        model:  string
        next_check_in:  string
        pending_locate:  bool
        requires_subscription:  bool
        serial_number: string
        tracking:  bool
        tracking_status:  string
    }

type Daily =
    {
        updated_at: string
        timestamp: string
        minutes_active: int
        activity_goal: int
        day_number: int
    }
    
type WEvent =
    {
        id: string
        timeline_id: string
        start_time: string
        end_time: string
        event_type: string
        event_title: string
        event_details: string option
        intensity: string
        minutes_active: int
        manually_created: string option
        share_url: string
        can_destroy: bool
        users: User[]
        current_user_present: bool
    }
    
type Highlight =
    {
        id: string
        timeline_id: string
        start_time: string
        event_type: string
        event_subtype: string
        event_details: string
        author : User
    }
    
type HourlyActivity =
    {
        activity : float option
        rest : float option
        distance : float option
        distance_units : string option
        calories: float option
    }
    
type DetailedDaily =
    {
        share_url : string
        events : WEvent[]
        date : string
        minutes_active: int
        minutes_rest : int
        calories : float
        distance : float
        distance_units : string
        time_zone : string
        last_updated_at : string
        hourly_activity : HourlyActivity[]
        day_number : int
    }
    
type Goal =
    {
        num_days : int
        goals_hit : int
        current_streak : int
        longest_streak : int
    }
    
type Averages =
    {
        average_minutes_active: float
        average_minutes_rest: float
        similar_dogs_minutes_active: float
        similar_dogs_minutes_rest: float
    }
    
type DailyTotal =
    {
        timestamp : string
        minutes_active : int
        minutes_rest : int
    }
    
type HighlightType =
    | Note
    | Food
    | Medication
    override x.ToString() = 
        match x with
        | Note -> "note"
        | Food -> "food"
        | Medication -> "medication"

// ---- Domain model END--------------------------------------

/// All other calls need the token returned
let GetToken authentication = 
    let json = Json.serialize(authentication)
    let response = 
        Http.RequestString(
            "https://app.whistle.com/api/tokens.json", 
            headers = [ HttpRequestHeaders.ContentType HttpContentTypes.Json ],
            body = TextRequest json
        )
    Json.deserialize<Token>(response).token
        
/// A generic GET that takes a type to define type-safe serialization. See the comment for raw JSON.
let AuthenticatedGet<'T> token url =
    let response = 
        Http.RequestString(
            url,
            headers = [ 
                HttpRequestHeaders.ContentType HttpContentTypes.Json
                "X-Whistle-AuthToken", token
                ]
        )
    //response //for raw json
    Json.deserialize<'T>(response) //for properly formatted object

/// Get the user for the device
let GetUser token =
    AuthenticatedGet<User> token "https://app.whistle.com/api/users/"

/// Get data for all dogs. Returns device ids and dog ids needed for other API calls
let GetDogs token = 
    AuthenticatedGet<Dog[]> token "https://app.whistle.com/api/dogs.json"
    
/// Get data for specific dog
let GetDog dogId token =
    AuthenticatedGet<Dog> token ("https://app.whistle.com/api/dogs/" + dogId)
    
/// Get status of Whistle Device
let GetDevice deviceId token = 
    AuthenticatedGet<Device> token ("https://app.whistle.com/api/devices/" + deviceId + ".json")

/// Returns minutes active and activity goal
let GetDailies dogId days token =
    AuthenticatedGet<Daily[]> token ("https://app.whistle.com/api/dogs/" + dogId + "/dailies?count=" + days.ToString())

/// More detailed dailies. The dailyId is returned in GetDailies
let GetDetailedDaily dogId dailyId token =
    AuthenticatedGet<DetailedDaily> token ("https://app.whistle.com/api/dogs/" + dogId + "/dailies/" + dailyId)
    
/// Returns specific WEvent. TimelineId is returned in DetailedDailies
let GetTimeline timelineId token =
    AuthenticatedGet<WEvent> token ("https://app.whistle.com/api/timeline/" +  timelineId)

/// Gets all highlights since the start of time
let GetHighlights dogId highlight token =
    AuthenticatedGet<Highlight[]> token ("https://app.whistle.com/api/dogs/" + dogId + "/highlights?type=" + highlight.ToString())

/// Returns empty, so can't test
let GetUsersPresent dogId token =
    AuthenticatedGet<User[]> token ("https://app.whistle.com/api/dogs/" + dogId + "/stats/users_present")
    
/// Get goals for this dog
let GetGoals dogId token =
    AuthenticatedGet<Goal> token ("https://app.whistle.com/api/dogs/" + dogId + "/stats/goals")
       
/// Returns average minutes active/rest vs similar dogs active/rest
let GetAverages dogId token =
    AuthenticatedGet<Averages> token ("https://app.whistle.com/api/dogs/" + dogId + "/stats/averages")

/// Returns minutes active and minutes rest. Fromdate format "2015-08-30"
let GetDailyTotals dogId fromDate token =
    AuthenticatedGet<DailyTotal[]> token ("https://app.whistle.com/api/dogs/" + dogId + "/stats/daily_totals/?start_time=" + fromDate )

## Using the API

The following calls demonstrate how to use the API.

In [72]:
//You need to insert your account information. The returned token is needed for following calls
let token = GetToken {email="YOUR WHISTLE EMAIL ADDRESS"; password="YOUR WHISTLE PASSWORD"} 
token |> printfn "%A\n"

token |> GetUser |> printfn "%A\n"
token |> GetDogs |> printfn "%A\n"

//Insert the dog id from the GetDogs call
token |> GetDog "5853" |> printfn "%A\n"
token |> GetHighlights "5853" HighlightType.Note |> printfn "%A\n"
token |> GetUserPresent "5853" |> printfn "%A\n"
token |> GetGoals "5853" |> printfn "%A\n"
token |> GetAverages "5853" |> printfn "%A\n"

//Insert the device id from the GetDogs call
token |> GetDevice "6439" |> printfn "%A\n"

//From date is the day you want, going forward to the current date
token |> GetDailyTotals "5853" "2019-11-30" |> printfn "%A\n"

//10 is the number of days you want, going backward from the current date
token |> GetDailies "5853" 10 |> printfn "%A\n"

//Insert a day number from the GetDailies call 
token |> GetDetailedDaily "5853" "18233" |> printfn "%A\n"

//Insert a timeline id from the GetDetailedDaily call
token |> GetTimeline "e486332483" |> printfn "%A\n"

"905aba472b54ea54ab4865d3ef027a02"

{id = "16938";
 first_name = "Andrew";
 last_name = "Olney";
 email = "aolney@memphis.edu";
 created_at = "2014-06-21T17:14:11Z";}

[|{id = "5853";
   city_id = "10913";
   date_of_birth = "2008-01-04";
   gender = "f";
   name = "Shiloh";
   weight = 62.0;
   weight_type = "pounds";
   age_in_months = 10;
   age_in_years = 11;
   breed_id = 432;
   breed_name = "German Shepherd Dog";
   average_minutes_active = 48.31318681;
   average_minutes_rest = 1050.5;
   average_calories = 1055.098798;
   average_distance = 2.84;
   distance_units = "miles";
   location = "Memphis, TN";
   device_id = "6439";
   device_serial = "W01-00683D";}|]

{id = "5853";
 city_id = "10913";
 date_of_birth = "2008-01-04";
 gender = "f";
 name = "Shiloh";
 weight = 62.0;
 weight_type = "pounds";
 age_in_months = 10;
 age_in_years = 11;
 breed_id = 432;
 breed_name = "German Shepherd Dog";
 average_minutes_active = 48.31318681;
 average_minutes_rest = 1050.5;
 average_calori